# Advent of Code 2018
*Phong Nguyen, December 2018*

This year is much harder than last year: more algorithmic problems and more complex simulation problems. Some problems I don't write code to return the answer but let it run intercept. Some problems I need help, or even code (day 23) from reddit. I didn't document much in the second half of the advent. I think I will split this file into topics. Probably together with other years.

### Some utility functions

In [1]:
import math
import re
import sys
from collections import deque, defaultdict, Counter, namedtuple
from itertools import chain, combinations
from heapq import heappop, heappush

flatten = chain.from_iterable

from numba import jit

def Input(day):
    "Return input file."
    return open('input{}.txt'.format(day))

def InputString(day):
    "Return the content of the input file as a string."
    return Input(day).read()

def InputRows(day):
    "Return the content of the input file as a list of string, each for a row."
    return InputString(day).splitlines()

def InputInts(day):
    "Return the content of the input file as a list of integers, each for a row."
    return [int(x) for x in InputString(day).splitlines()]

def ints(start, end, step=1): return range(start, end+1, step)

def get_ints(text):
    return list(map(int, re.findall(r'-?\d+', text)))

# 2D points
UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def add_tuples(t1, t2):
     return tuple(sum(x) for x in zip(t1, t2))
        
def mht(x, y):
    return abs(x[0] - y[0]) + abs(x[1] - y[1])

def Mht_distance(p):
    return abs(p[0]) + abs(p[1])

def argmax(a):
    return a.index(max(a))

def argmax_dict(d):
    return max(d, key=(lambda k: d[k]))

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def breadth_first(start, goal, moves_func):
    "Find a shortest sequence of states from start to the goal."
    frontier = deque([start]) # A queue of states
    previous = {start: None}  # start has no previous state; other states will
    while frontier:
        s = frontier.popleft()
        if s == goal:
            return path(previous, s)
        for s2 in moves_func(s):
            if s2 not in previous:
                frontier.append(s2)
                previous[s2] = s
                
def path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return [] if (s is None) else path(previous, previous[s]) + [s]

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.

    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
                
    return dict(fail=True, front=len(frontier), prev=len(previous))

%load_ext autoreload
%autoreload 2

## Day 1: Chronal Calibration
[Problem Description](https://adventofcode.com/2018/day/1)

In [2]:
def day1a(nums):
    return sum(nums)

nums = InputInts(1)
day1a(nums)

505

In [3]:
def day1b(nums):
    seen = set()
    f = 0
    while True:
        for x in nums:
            f += x
            if f in seen:
                return f
            else:
                seen.add(f)
        
day1b(nums)

72330

## Day 2: Inventory Management System
[Problem Description](https://adventofcode.com/2018/day/2)

In [4]:
def has_two(text):
    return 2 in Counter(text).values()

def has_three(text):
    return 3 in Counter(text).values()
    
def day2a(lines):
    two = sum(1 for l in lines if has_two(l))
    three = sum(1 for l in lines if has_three(l))
    return two * three

lines = InputRows(2)
day2a(lines)

8892

In [5]:
def are_correct(t1, t2):
    diff_count = 0
    for i in range(len(t1)):
        if t1[i] != t2[i]:
            diff_count += 1
        if diff_count > 1:
            return False
    return diff_count == 1
    
def day2b(lines):
    for c in combinations(lines, 2):
        if are_correct(*c):
            t1, t2 = c
            common_string = ''
            for i in range(len(t1)): 
                if t1[i] == t2[i]: 
                    common_string += t1[i]
            print(common_string)
                
%time day2b(lines)

zihwtxagifpbsnwleydukjmqv
CPU times: user 38.3 ms, sys: 1.29 ms, total: 39.6 ms
Wall time: 38.6 ms


A shorter way (less code) and also easier to find the common string for the answer is to remove each character and check if the rest of the two strings are the same. It is slower though.

In [6]:
%%time

for c in combinations(lines, 2):
    t1, t2 = c
    for i in range(len(t1)):
        x1 = t1[:i] + t1[i+1:]
        x2 = t2[:i] + t2[i+1:]
        if x1 == x2:
            print(x1)
            break

zihwtxagifpbsnwleydukjmqv
CPU times: user 631 ms, sys: 8.94 ms, total: 640 ms
Wall time: 634 ms


Reading solutions in reddit, I see using `zip` is a neat way of looping through pairs of things. Also, a more pythonic way of summing booleans. So, rewrite my first solution.

In [7]:
def are_correct_zip(t1, t2):
    return sum(c1 != c2 for c1, c2 in zip(t1, t2)) == 1
    
def day2b_zip(lines):
    for c in combinations(lines, 2):
        if are_correct(*c):
            common_string = ''.join(c1 for c1, c2 in zip(*c) if c1 == c2)
            print(common_string)
                
%time day2b_zip(lines)

zihwtxagifpbsnwleydukjmqv
CPU times: user 39.9 ms, sys: 1.7 ms, total: 41.6 ms
Wall time: 40.2 ms


Well, it is as fast as the original one, even though it needs to sum up all differences, without early exit.

## Day 3: No Matter How You Slice It
[Problem Description](https://adventofcode.com/2018/day/3)

I started with a bad algorithm, checking each pixel against all claims. There are 1 millions pixel, so too slow. Then, I switch to a more active approach: recording the area of each claim and check how many pixels are recorded more than twice. As the size of a claim is small, this approach is much faster.

In [8]:
def parse_claim(line):
    "Return x, y, w, h"
    nums = re.findall(r'\d+', line)[1:]
    return tuple(map(int, nums))

def day3(lines):
    claims = list(map(parse_claim, lines))
    lookup = defaultdict(int)

    for c in claims:
        for x in range(c[0], c[0] + c[2]):
            for y in range(c[1], c[1] + c[3]):
                lookup[(x, y)] += 1
    
    print('part 1', sum(c > 1 for c in lookup.values()))
    
    def is_overlapped(c):
        for x in range(c[0], c[0] + c[2]):
            for y in range(c[1], c[1] + c[3]):
                if lookup[(x, y)] > 1:
                    return True

    for i, c in enumerate(claims):
        if not is_overlapped(c):
            print('part 2', i + 1)

day3(InputRows(3))

part 1 115304
part 2 275


## Day 4: Repose Record
[Problem Description](https://adventofcode.com/2018/day/4)

In [9]:
def day4(rows):
    rows = sorted(rows)
    records = []
    total_minutes_lookup = defaultdict(int)
    minute_lookup = defaultdict(Counter)
    start = None
    
    for row in rows:
        nums = list(map(int, re.findall(r'\d+', row)))
        if len(nums) == 6:
            guard = nums[5]
        else:
            month, day, hour, minute = nums[1:5]
            if start == None:
                start = minute
            else:
                stop = minute
                total_minutes_lookup[guard] += (stop - start)
                minute_lookup[guard].update(range(start, stop))
                start = None
        
    most_sleep_guard = argmax_dict(total_minutes_lookup)
    most_minute = minute_lookup[most_sleep_guard].most_common(1)[0][0]
    print('part 1', most_sleep_guard * most_minute)

    max_all = [(guard, *minute_lookup[guard].most_common(1)[0]) for guard in minute_lookup]
    guard, minute, _ = max(max_all, key=(lambda a:a[2]))
    print('part 2', guard * minute)

day4(InputRows(4))

part 1 84636
part 2 91679


## Day 5: Repose Record
[Problem Description](https://adventofcode.com/2018/day/5)

In [11]:
p = re.compile(r'aA|Aa|bB|Bb|cC|Cc|dD|Dd|eE|Ee|fF|Ff|gG|Gg|hH|Hh|iI|Ii|jJ|Jj|kK|Kk|lL|Ll|mM|Mm|nN|Nn|oO|Oo|pP|Pp|qQ|Qq|rR|Rr|sS|Ss|tT|Tt|uU|Uu|vV|Vv|wW|Ww|xX|Xx|yY|Yy|zZ|Zz')
    
def replace_polymer(text):
    return re.sub(p, '', text)

def count_length(text):
    new_text = replace_polymer(text)
    while new_text != text:
        text = new_text
        new_text = replace_polymer(text)
        
    return len(text)

def day5(text):
    alls = []
    for x in 'qwertyuiopasdfghjklzxcvbnm':
        t = text.replace(x, '').replace(x.upper(), '')
        alls.append((count_length(t), x))
    return min(alls)

day5(InputString(5))

(6550, 'x')

Under pressure, I can't think of an elegant method. Fortunately, this regex replacement is not too slow. I know I shouldn't use string replacement. Now, more relaxed, because we don't need the string, just the final length, or the number of remove operations.

Hmm, it's not easy as I think because when a match is removed, the cursor needs to go back one. A stack data structure is more suitable.

In [12]:
def match(a, b):
    return a != b and a.lower() == b.lower()

def count_length(text):
    stack = []
    for s in text:
        if stack and match(stack[-1], s):
            stack.pop()
        else:
            stack.append(s)
    return len(stack)

def day5x(text):
    return count_length(text)

day5x(InputString(5))

9172

## Day 6: Chronal Coordinates
[Problem Description](https://adventofcode.com/2018/day/6)

Well, the most tricky day so far. For part 1, I had to submit at least 5 times. I didn't write code to detect infinite regions, instead, relying on scanning through the output. Finite regions are the ones that don't change when you extend the boundary.

In [13]:
def mht(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

In [14]:
def exclude(a, min_x, max_x, min_y, max_y):
    return [p for p in a if p[0][0] > min_x and p[0][0] < max_x and p[0][1] > min_y and p[0][1] < max_y]

def day6(rows):
    coords = []
    lookup = defaultdict(int)
    for row in rows:
        p = tuple(map(int, row.split(',')))
        coords.append(p)
    
    min_x = min(coords, key=lambda x:x[0])[0]
    max_x = max(coords, key=lambda x:x[0])[0]
    min_y = min(coords, key=lambda x:x[1])[1]
    max_y = max(coords, key=lambda x:x[1])[1]
    print(min_x, max_x, min_y, max_y)
    for i in range(min_x-200, max_x+400):
        for j in range(min_y-200, max_y+400):
            distances = [(c, mht(c, (i, j))) for c in coords]
#             print(distances)
            min_distance = min(distances, key=lambda x: x[1])
#             print(min_distance)
            count = 0
            p = None
            for d in distances:
                if d[1] == min_distance[1]:
                    p = d[0]
                    count += 1
            if count == 1:
                lookup[p] += 1
                

    sorted_by_value = sorted(lookup.items(), key=lambda kv: kv[1], reverse=True)
#     print(sorted_by_value)
    return exclude(sorted_by_value, min_x, max_x, min_y, max_y)

In [15]:
day6(InputRows(6))

41 353 43 358


[((348, 53), 126085),
 ((78, 356), 110400),
 ((67, 53), 67800),
 ((304, 190), 29318),
 ((106, 338), 28839),
 ((224, 339), 28827),
 ((339, 256), 27582),
 ((64, 294), 19800),
 ((202, 86), 18620),
 ((65, 203), 16359),
 ((61, 123), 10754),
 ((336, 303), 9390),
 ((336, 283), 9142),
 ((91, 234), 6177),
 ((328, 72), 6114),
 ((236, 337), 5971),
 ((166, 169), 5532),
 ((159, 270), 5472),
 ((277, 110), 3903),
 ((95, 85), 3743),
 ((114, 146), 3118),
 ((234, 119), 2797),
 ((236, 191), 2686),
 ((229, 286), 2236),
 ((114, 260), 1789),
 ((115, 256), 1772),
 ((310, 234), 1641),
 ((242, 164), 1533),
 ((226, 298), 1526),
 ((225, 179), 1490),
 ((281, 232), 1485),
 ((263, 187), 1462),
 ((285, 279), 1414),
 ((82, 142), 1294),
 ((244, 318), 1284),
 ((310, 295), 1028),
 ((301, 335), 981),
 ((261, 267), 822),
 ((321, 335), 693),
 ((329, 305), 418),
 ((100, 143), 406),
 ((266, 253), 406),
 ((97, 140), 320),
 ((261, 258), 286),
 ((312, 341), 276)]

Part 2 sucks. Very easy but rejected by the site. And it looks like there's a bug there.

In [17]:
def day6(rows, limit):
    coords = []
    lookup = defaultdict(int)
    for row in rows:
        p = tuple(map(int, row.split(',')))
        coords.append(p)
    
    min_x = min(coords, key=lambda x:x[0])[0]
    max_x = max(coords, key=lambda x:x[0])[0]
    min_y = min(coords, key=lambda x:x[1])[1]
    max_y = max(coords, key=lambda x:x[1])[1]

    print(min_x, max_x, min_y, max_y)
    count = 0
    for i in range(min_x-100, max_x+100):
        for j in range(min_y-100, max_y+100):
            distance = sum(mht(c, (i, j)) for c in coords)
            if distance < limit:
                count += 1
    return count

day6(InputRows(6), 10000)

41 353 43 358


36216

## Day 7: The Sum of Its Parts
[Problem Description](https://adventofcode.com/2018/day/7)

In [18]:
def day7a(rows):
    g = defaultdict(list)
    parent = defaultdict(list)
    for r in rows:
        o, d = r[5], r[36]
        g[o].append(d)
        parent[d].append(o)
        
    left = sorted(set(list(g.keys()) + list(flatten(g.values()))))
    avails = set(c for c in left if not parent[c])
#     print(avails)
    path = []
    while True:
        for n in left:
            if n in avails:
                path.append(n)
                left.remove(n)
                for c in g[n]:
                    if all(p in path for p in parent[c]):
                        avails.add(c)
                if not left:
                    return ''.join(path)
#                 print('add ', n)
#                 print('avails', avails)
                break
    
day7a(InputRows(7))

'JNOIKSYABEQRUVWXGTZFDMHLPC'

In [19]:
alpha = sorted('QWERTYUIOPASDFGHJKLZXCVBNM')

def day7b(rows):
    g = defaultdict(list)
    parent = defaultdict(list)
    for r in rows:
        o, d = r[5], r[36]
        g[o].append(d)
        parent[d].append(o)
        
    left = set(sorted(set(list(g.keys()) + list(flatten(g.values())))))
    avails = set(c for c in left if not parent[c])
    started = set()
    workers = [0, 0, 0, 0, 0]    
    count = 0
    running = dict()

    # Is any task available?
    while left:
        not_started = avails - started
#         print(not_started)
        if not_started:
            for m in sorted(not_started):
                # Is any worker available?
                for i in range(len(workers)):
                    # Yes, assign task with corresponding time
                    if not workers[i]:
                        workers[i] = running[m] = alpha.index(m) + 61
                        started.add(m)
                        break # Just need to assign once
        
#         print(count, running, workers, not_started)
        
        # Time ticking
        for i in range(len(workers)):
            if workers[i]:
                workers[i] -= 1
        to_delete = []
        for m in running:
            running[m] -= 1
            # A task has finished?
            if running[m] == 0:
                to_delete.append(m)
                left.remove(m)
                
                # Check more available tasks: they can only come from its children
                for c in g[m]:
                    if all(p not in left for p in parent[c]):
                        avails.add(c)

        for t in to_delete:
            del running[t]

        count += 1

    return count
    
day7b(InputRows(7))

1099

In [20]:
def day7a2(rows):
    g = defaultdict(list)
    parent = defaultdict(list)
    for r in rows:
        o, d = r[5], r[36]
        g[o].append(d)
        parent[d].append(o)
        
    left = set(sorted(set(list(g.keys()) + list(flatten(g.values())))))
    avails = set(c for c in left if not parent[c])
    path = []

    while left:
        not_started = avails - set(path)
        if not_started:
            m = min(not_started)
            path.append(m)
            left.remove(m)
                
            # Check more available tasks: they can only come from its children
            for c in g[m]:
                if all(p not in left for p in parent[c]):
                    avails.add(c)

    return ''.join(path)
    
day7a2(InputRows(7))
print(day7a2(InputRows(7)) == day7a(InputRows(7)))

True


# Day 8: Memory Maneuver
[Problem Description](https://adventofcode.com/2018/day/8)

Python doesn't allow to modify global variables. So, first I have to cheat it with modifying attributes of a dictionary.

In [21]:
def day8(rows):
    root = {}
    tmp = { 'idx': 0, 'sum_meta': 0 }
    idx = 0
    sum_meta = 0
    
    def parse_child(tmp):
        "Return a child node"
        num_children = rows[tmp['idx']]
        tmp['idx'] += 1
        num_meta = rows[tmp['idx']]
        tmp['idx'] += 1
        
        child_values = [parse_child(tmp) for i in range(num_children)]
        
        metas = rows[tmp['idx']:tmp['idx']+num_meta]
        tmp['idx'] += num_meta
        tmp['sum_meta'] += sum(metas)
            
        if num_children == 0:
            return sum(metas)
        else:
            value = 0
            for m in metas:
                if 1 <= m <= num_children:
                    value += child_values[m - 1]
            return value
    
    value = parse_child(tmp)
    
    return tmp['sum_meta'], value
    
day8(list(map(int, InputString(8).split(' '))))

(47647, 23636)

Then, looking at reddit to see how people normally avoid accessing global variables: use returning values and input, use a tuple to return multiple values. Not very comfortable doing so. Some people also cheat by modifying an array (the sum of meta)

In [22]:
def day8(nums):
    def parse_child(i):
        num_children, num_meta = nums[i: i + 2]
        i += 2

        # Recursive
        sum_meta = 0
        values = []
        for _ in range(num_children):
            i, meta, value = parse_child(i)
            sum_meta += meta
            values.append(value)

        # Main
        metas = nums[i:i + num_meta]
        sum_meta += sum(metas)
        i += num_meta

        if num_children == 0:
            return i, sum_meta, sum_meta
        else:
            value = sum(values[m - 1] for m in metas if 1 <= m <= num_children)
            return i, sum_meta, value
    
    return parse_child(0)[1:]

day8(list(map(int, InputString(8).split(' '))))

(47647, 23636)

# Day 9: Marble Mania
[Problem Description](https://adventofcode.com/2018/day/9)

I woke up half an hour late today. Fortunately, still manage the top 1500 (1423/1223) as other rough days.

The first part is list-based. Apparently, too slow for part 2.

In [23]:
def deal(num_players, last_marble):
    circle = [0]
    current_idx = 0
    players = defaultdict(int)
    player_idx = 0
    
    for m in ints(1, last_marble):
        if m % 23:
            if current_idx == len(circle) - 1:
                circle = circle[:1] + [m] + circle[1:]
                current_idx = 1
            else:
                circle = circle[:current_idx + 2] + [m] + circle[current_idx + 2:]
                current_idx = current_idx + 2
#             print(circle, circle[current_idx])
        else:
            extra_idx = (current_idx - 7) % len(circle) 
            players[player_idx] += m + circle[extra_idx]
            del circle[extra_idx]
            current_idx = extra_idx % len(circle)
#             print(circle, circle[current_idx])
            
        player_idx = (player_idx + 1) % num_players
        
    return players[max(players, key=players.get)]

def day9(s):
    print(deal(*list(map(int, re.findall(r'\d+', s)))))
  
day9('5 players; last marble is worth 25 points')
# day9('432 players; last marble is worth 7101900 points')

32


I don't know any linked list class in Python. When I googled, I saw some a basic Node class, then use it. I did a lot prev/next pointers and so surprised that I didn't end up with infinite loop!!

In [24]:
class Node:
    def __init__(self, val):
        self.val = val
        self.next = None
        self.prev = None

In [25]:
def print_circle(current):
    zero = current
    while zero.val != 0:
        zero = zero.next
    n = zero
    s = '0 '
    while n.next != zero:
        x = ('(' + str(n.next.val) + ')') if n.next == current else str(n.next.val)
        s += x + ' '
        n = n.next
    print(s)

In [26]:
%%time

def deal(num_players, last_marble):
    current = Node(0)
    current.next = current
    current.prev = current
        
    players = defaultdict(int)
    player_idx = 0
    
    for m in ints(1, last_marble):
        if m % 23:
            n = Node(m)
            current.next.next.prev = n
            n.next = current.next.next
            current.next.next = n
            n.prev = current.next
            current = n
        else:
            extra = current.prev.prev.prev.prev.prev.prev.prev
            current = extra.next
            players[player_idx] += m + extra.val
            extra.prev.next = extra.next
            
#         print_circle(current)
        player_idx = (player_idx + 1) % num_players
        
    return max(players.values())

def day9(s):
    print(deal(*list(map(int, re.findall(r'\d+', s)))))
  
# day9('5 players; last marble is worth 25 points')
day9('432 players; last marble is worth 7101900 points')

3338341690
CPU times: user 21.4 s, sys: 477 ms, total: 21.9 s
Wall time: 22 s


Then reddit shows `deque` with `rotate` method. I knew this class but didn't think of using `rotate` to manipulate.

In [27]:
%%time

def deal2(num_players, last_marble):
    circle = deque([0])
        
    players = defaultdict(int)
    player_idx = 0
    
    for m in ints(1, last_marble):
        if m % 23:
            circle.rotate(-1)
            circle.append(m)
        else:
            circle.rotate(7)
            players[player_idx] += m + circle.pop()
            circle.rotate(-1)
            
        player_idx = (player_idx + 1) % num_players
        
    return max(players.values())

def day9x(s):
    print(deal2(*list(map(int, re.findall(r'\d+', s)))))
  
# day9('5 players; last marble is worth 25 points')
day9('432 players; last marble is worth 7101900 points')

3338341690
CPU times: user 22.1 s, sys: 605 ms, total: 22.7 s
Wall time: 22.7 s


Both solutions are about the same time, 20s. But if I run the code directly in Python, not in this notebook, it just takes 3s.

# Day 10: The Stars Align
[Problem Description](https://adventofcode.com/2018/day/10)

In [28]:
def day10(rows):
    points = []
    for row in rows:
        nums = get_ints(row)
        points.append(nums)

    for k in range(20000):
        min_x = min(p[0] for p in points)
        min_y = min(p[1] for p in points)
        max_x = max(p[0] for p in points)
        max_y = max(p[1] for p in points)
        
        threshold = 20
        if max_y - min_y < threshold:
            print(k)
            coords = [(p[0], p[1]) for p in points]
            s = ''
            for y in ints(min_y, max_y):
                for x in ints(min_x, max_x):
                    s += '*' if (x, y) in coords else ' '
                s += '\n'
            print(s)
        
        for p in points:
            p[0] += p[2]
            p[1] += p[3]
    
day10(InputRows(10))

10680
            **                                                     *    
* *   *   ***                * *          *              * *            
                    *       ** *      *  *  *   *          *            
    *   * *  **            *          * *  *               *     *      
             *               *   *          *        *         *     *  
             *      *      **       * * **   **   **      *        *   *
  *    * *        * *     * *   * *** *          *   * *  * * * **  ** *
   *     *      *    * *    * *       *****       *** *     * *         
  *     *     **         **  **  *     ** *    ** *       ** *   ** *   
      *   ***    **  *      ** *  *   *        *   **   * ** **    *    
        **   *** ** **** ** *   *** * *   * *  * ***        *  * **   * 
   *  * *   **   ***    * * *    *   * *   **   **      *  ***     *    
   **      *      *  **   * * *  *   ****    *      * * * * * *  *      
    *   *   * *            *  * **  * *  * **

# Day 11: Chronal Charge
[Problem Description](https://adventofcode.com/2018/day/11)

In [31]:
def day11(n):
    def hundred(x): return int(str(x)[-3])

    def power(x, y):
        a = ((x + 10) * y + n) * (x + 10)
        return hundred(a) - 5

    grid = {}
    s = 300
    for x in ints(1, s):
        for y in ints(1, s):
            grid[(x, y)] = power(x, y)

    ans = {}
    for k in ints(1, 300):
        powers = {}
        for x in ints(1, s - k + 1):
            for y in ints(1, s - k + 1):
                total = 0
                for i in range(k):
                    for j in range(k):
                        total += grid[(x+i, y+j)]
                powers[(x, y)] = total
        max_key = max(powers, key=lambda x: powers[x])
        ans[(*max_key, k)] = powers[max_key]
        print(*max_key, k, powers[max_key])
    max_key = max(ans, key=lambda x: ans[x])
    return max_key

day11(5791)

1 2 1 4
21 193 2 16
20 68 3 29
20 18 4 39
241 281 5 52
240 281 6 54
241 273 7 67
241 273 8 63
237 281 9 87
237 281 10 94
235 279 11 110
235 277 12 109
234 279 13 107
235 275 14 104
231 274 15 108
231 273 16 111
231 272 17 106


KeyboardInterrupt: 

# Day 12: Subterranean Sustainability 
[Problem Description](https://adventofcode.com/2018/day/12)

In [ ]:
def print_pots(pots):
    start = min(pots.keys())
    end = max(pots.keys())
#     print(start, end)
    s = [pots[i] for i in ints(start, end)]
    print(start, ''.join(s))
            
def plant(s):
    return '#' if s else '.'

def day12(rows, state):
    rules = set(rows)
    pots = {}
    for i, c in enumerate(state):
        pots[i] = c
        
    print_pots(pots)
    
    for idx in range(1000):
        start = min(pots.keys()) - 2 
        end = len(pots) + 2
        temp = []
        for i in range(start, end):
            s = pots.get(i-2, '.') + pots.get(i-1, '.') + pots.get(i, '.') + pots.get(i+1, '.') + pots.get(i+2,'.')
#             print(s)
            temp.append('#' if s in rules else '.')
        for i in range(start, end):
            pots[i] = temp[i-start]
#         print(pots)
#         print_pots(pots)

        total = 0
        for k in pots:
            if pots[k] == '#':
                total += k
        if (idx + 1) % 100 == 0:
            print(idx + 1, total)
    

# day12(InputRows(12), '#..#.#..##......###...###')
day12(InputRows(12), '##..##....#.#.####........##.#.#####.##..#.#..#.#...##.#####.###.##...#....##....#..###.#...#.#.#.#')


# Day 13: Mine Cart Madness
[Problem Description](https://adventofcode.com/2018/day/13)

This cost me plenty of time as my code is incorrect because of the collision detection. I thought that all carts move at the same time and will collide if any in the new frame. I thought the moving top/down left/right just for explanation and doesn't affect. It turned out that a cart can collide with carts in previous tick. Look like no one makes the same mistake like me in reddit :(.

In [ ]:
def day13(rows):
    carts = []
    dir_map = { '<': 'left', '>': 'right', 'v': 'down', '^': 'up' }
    dirs = { 'left': (-1, 0), 'right': (1, 0), 'down': (0, 1), 'up': (0, -1) }
    turns = {}
    inters = set()
    switch = {
        '\\': {
            'left': 'up',
            'right': 'down',
            'down': 'right',
            'up': 'left'
        },
        '/': {
            'left': 'down',
            'right': 'up',
            'down': 'left',
            'up': 'right'
        }
    }
    inter_switch = {
        'left': {
            'left': 'down',
            'right': 'up',
            'down': 'right',
            'up': 'left'
        },
        'right': {
            'left': 'up',
            'right': 'down',
            'down': 'left',
            'up': 'right'
        },
        'straight': {
            'left': 'left',
            'right': 'right',
            'down': 'down',
            'up': 'up'
        }
    }
    for i, row in enumerate(rows):
        for j, c in enumerate(row):
            if c in '<>v^':
                carts.append([j, i, dir_map[c], 'left'])
            if c in '\/':
                turns[(j, i)] = c
            if c == '+':
                inters.add((j, i))
                
    carts = sorted(carts, key=lambda x: (x[1], x[0]))
#     print(carts)
    while True:
        if len(carts) == 1:
            return carts[0][0], carts[0][1]
        carts = sorted(carts, key=lambda x: (x[1], x[0]))
        for c in carts:
            step = dirs[c[2]]
            next_coord = c[0] + step[0], c[1] + step[1]
            if next_coord in turns:
                c[2] = switch[turns[next_coord]][c[2]]
            elif next_coord in inters:
                c[2] = inter_switch[c[3]][c[2]]
                if c[3] == 'left':
                    c[3] = 'straight'
                elif c[3] == 'straight':
                    c[3] = 'right'
                elif c[3] == 'right':
                    c[3] = 'left'
            if next_coord in [(c[0], c[1]) for c in carts]:
#                 return next_coord
                carts = [nc for nc in carts if (nc[0], nc[1]) not in [next_coord, (c[0], c[1])]]
            c[0] += step[0]
            c[1] += step[1]
    
day13(InputRows(13))

# Day 14: Chocolate Charts
[Problem Description](https://adventofcode.com/2018/day/14)

In [ ]:
def day14(n):
    def go(x, y):
        return (x+y) % len(nums)
    
    first, second = 0, 1
    nums = [3, 7]
    while len(nums) < n + 10:
        s = nums[first] + nums[second]
        if s < 10:
            nums.append(s)
        else:
            nums.append(1)
            nums.append(s % 10)
        first = go(first, 1 + nums[first])
        second = go(second, 1 + nums[second])
    return ''.join(map(str, nums[n:n+10]))

day14(430971)

In [ ]:
def day14b(n):
    def go(x, y):
        return (x+y) % len(nums)
    
    first, second = 0, 1
    nums = [3, 7]
    seq = list(map(int, n))
    
    while True:
        s = nums[first] + nums[second]
        if s < 10:
            nums.append(s)
        else:
            nums.append(1)
            nums.append(s % 10)
        first = go(first, 1 + nums[first])
        second = go(second, 1 + nums[second])
        
        if nums[-len(seq):] == seq:
            return len(nums) - len(seq)
        if nums[-len(seq)-1:-1] == seq:
            return len(nums) - len(seq) - 1

day14b('430971')

# Day 15: Beverage Bandits
[Problem Description](https://adventofcode.com/2018/day/15)

A very sophisticated simulation problem. I don't even want to clean up the code after get the stars.

In [ ]:
def day15(rows):
    power = 25

    def show():
        s = ''
        ges = sorted(goblins + elves)
        for i, row in enumerate(M):
            s += ''.join(row) + ' '
            for j in range(len(row)):
                for g in ges:
                    if g[0] == (i, j):
                        s += g[2] + '(' + str(g[1]) +') '
                
            s += '\n'
            
        print(s)
        
    def neighbors4(point): 
        "The four neighbors (without diagonals)."
        x, y = point
        return ((x-1, y), (x, y-1), (x, y+1), (x+1, y))
        
    def moves(x):
#         print([m for m in neighbors4(x) if M[m[0]][m[1]] == '.'])
        return [m for m in neighbors4(x) if M[m[0]][m[1]] == '.']

    goblins = [] 
    elves = []
    M = []
    for i, r in enumerate(rows):
        row = []
        for j, c in enumerate(r):
            row.append(c)
            if c == 'G':
                goblins.append([(i, j), 200, 'G'])
            if c == 'E':
                elves.append([(i, j), 200, 'E'])
        M.append(row)
    ges = sorted(goblins + elves)
    
#     m = ges[0]
#     enemies = [e for e in ges if e[2] != m[2]]
#     e = enemies[0]
#     print(m[0], '->' , e[0])
#     M[e[0][0]][e[0][1]] = '.'
#     return breadth_first(m[0], e[0], moves)

    def breadth_first(start, goals, moves_func):
        "Find a shortest sequence of states from start to the goal."
        frontier = deque([start]) # A queue of states
        previous = {start: None}  # start has no previous state; other states will
        all_paths = []
        while frontier:
            s = frontier.popleft()
            if s in goals:
                all_paths.append(path(previous, s))
            for s2 in moves_func(s):
                if s2 not in previous:
                    frontier.append(s2)
                    previous[s2] = s
        return all_paths

    def attack(m):
        targets = []
        for r, c in neighbors4(m[0]):
            if M[r][c] in 'GE' and M[r][c] != m[2]:
                for e in enemies:
                    if e[0] == (r, c):
                        targets.append(e)
        if targets:
            e = min(targets, key=lambda x: (x[1], x[0]))
            
            e[1] -= (power if m[2] == 'E' else 3)
#                 print(m, 'att', e)
            if e[1] <= 0:
#                     print('die', r,c , M[r][c], e)
                r, c = e[0]
#                 print('before')
#                 show()
                M[r][c] = '.'
#                 print('after')
#                 show()
                if e in goblins:
                    goblins.remove(e)
                if e in elves:
                    elves.remove(e)
                if e in enemies:
                    enemies.remove(e)
        return targets

#     show()
#     print()

    elves_count = len(elves)
        
    count = 0
    while True:
        count += 1
        ges = sorted(goblins + elves)
        for m in ges:
            if m not in goblins and m not in elves:
                continue
                
            enemies = [e for e in ges if e[2] != m[2] and e[1] > 0]
#             print('to consider', enemies)
            # Find shortest path to each enemy
#             print('I am ', m[2], m[0])

            if not attack(m):
#                 ges2 = sorted(goblins + elves)
#                 enemies = [e for e in ges if e[2] != m[2]]
#                 print('new ', enemies)
#                 for e in enemies:
# #                     print('find', m[0], e[0])
#                     # Note: move to near by targets, not the tartgets themselves
#                     for n in moves(e[0]):
#                         # Mark the target open first
# #                         M[e[0][0]][e[0][1]] = '.'
#                         apath = breadth_first(m[0], n, moves)
#                         if apath:
#                             paths += apath
#                         # Reset
# #                         M[e[0][0]][e[0][1]] = e[2]
# #                 print(m[0], e[0], paths)

                targets = []
                for e in enemies:
                    for n in moves(e[0]):
                        if n not in targets:
                            targets.append(n)
                
#                 print(targets)            
                paths = breadth_first(m[0], targets, moves)
                
                if paths:
                    shortest_dist = min(list(map(len, paths)))
                    target = min([p[-1] for p in paths if len(p) == shortest_dist])
                
                
                    # the first target in reading order is the chosen one
#                     target = min(p[-1] for p in paths)
#                     print('choose', target)

                    paths = breadth_first(m[0], [target], moves)
                    
                    if paths:
                        p = min(paths, key=lambda x: x[1])
#                         print('from', m[0], 'to', p)
                        
                        next_move = p[1]
                        # Make a move
                        old_pos = m[0]
                        M[old_pos[0]][old_pos[1]] = '.'
                        m[0] = next_move
                        M[next_move[0]][next_move[1]] = m[2]
        #                 print('...', m[2], old_pos, next_move)

                        attack(m)
    #             print()
          
            
#         if len(elves) < elves_count:
#             print('dead')
#             return
        
#         print('After', count, 'rounds')
#         show()
#         print()

            
        if not goblins or not elves:
            sumg = sum(g[1] for g in goblins)
            sume = sum(g[1] for g in elves)
            return count, count * (sumg + sume), (count - 1) * (sumg + sume)

day15(InputRows(15))

# Day 16: Chronal Classification
[Problem Description](https://adventofcode.com/2018/day/16)

In [ ]:
def day15(rows):
    power = 25

    def show():
        s = ''
        ges = sorted(goblins + elves)
        for i, row in enumerate(M):
            s += ''.join(row) + ' '
            for j in range(len(row)):
                for g in ges:
                    if g[0] == (i, j):
                        s += g[2] + '(' + str(g[1]) +') '
                
            s += '\n'
            
        print(s)
        
    def neighbors4(point): 
        "The four neighbors (without diagonals)."
        x, y = point
        return ((x-1, y), (x, y-1), (x, y+1), (x+1, y))
        
    def moves(x):
#         print([m for m in neighbors4(x) if M[m[0]][m[1]] == '.'])
        return [m for m in neighbors4(x) if M[m[0]][m[1]] == '.']

    goblins = [] 
    elves = []
    M = []
    for i, r in enumerate(rows):
        row = []
        for j, c in enumerate(r):
            row.append(c)
            if c == 'G':
                goblins.append([(i, j), 200, 'G'])
            if c == 'E':
                elves.append([(i, j), 200, 'E'])
        M.append(row)
    ges = sorted(goblins + elves)
    
#     m = ges[0]
#     enemies = [e for e in ges if e[2] != m[2]]
#     e = enemies[0]
#     print(m[0], '->' , e[0])
#     M[e[0][0]][e[0][1]] = '.'
#     return breadth_first(m[0], e[0], moves)

    def attack(m):
        targets = []
        for r, c in neighbors4(m[0]):
            if M[r][c] in 'GE' and M[r][c] != m[2]:
                for e in enemies:
                    if e[0] == (r, c):
                        targets.append(e)
        if targets:
            e = min(targets, key=lambda x: (x[1], x[0]))
            
            e[1] -= (power if m[2] == 'E' else 3)
#                 print(m, 'att', e)
            if e[1] <= 0:
#                     print('die', r,c , M[r][c], e)
                r, c = e[0]
#                 print('before')
#                 show()
                M[r][c] = '.'
#                 print('after')
#                 show()
                if e in goblins:
                    goblins.remove(e)
                if e in elves:
                    elves.remove(e)
                if e in enemies:
                    enemies.remove(e)
        return targets

#     show()
#     print()

    elves_count = len(elves)
        
    count = 0
    while True:
        count += 1
        ges = sorted(goblins + elves)
        for m in ges:
            if m not in goblins and m not in elves:
                continue
                
            enemies = [e for e in ges if e[2] != m[2] and e[1] > 0]
#             print('to consider', enemies)
            # Find shortest path to each enemy
            paths = []
#             print('I am ', m[2], m[0])

            if not attack(m):
#                 ges2 = sorted(goblins + elves)
#                 enemies = [e for e in ges if e[2] != m[2]]
#                 print('new ', enemies)
                
                for e in enemies:
#                     print('find', m[0], e[0])
                    # Mark the target open first
                    M[e[0][0]][e[0][1]] = '.'
                    apath = breadth_first(m[0], e[0], moves)
                    if apath:
                        paths.append(breadth_first(m[0], e[0], moves))
                    # Reset
                    M[e[0][0]][e[0][1]] = e[2]
#                 print(m[0], e[0], paths)

                if not paths:# No way to go
                    continue
        
                p = min(paths, key=lambda x: (len(x), x[1]))

                next_move = p[1]
                # Make a move
                old_pos = m[0]
                M[old_pos[0]][old_pos[1]] = '.'
                m[0] = next_move
                M[next_move[0]][next_move[1]] = m[2]
#                 print('...', m[2], old_pos, next_move)
                
                attack(m)
#             print()
          
            
        if len(elves) < elves_count:
            print('dead')
            return
        
        print('After', count, 'rounds')
        show()
        print()

            
        if not goblins or not elves:
            sumg = sum(g[1] for g in goblins)
            sume = sum(g[1] for g in elves)
            return count,sumg + sume, count * (sumg + sume), (count - 1) * (sumg + sume)
    
day15(InputRows(15))

In [ ]:
def day15(rows):
    power = 25

    def show():
        s = ''
        ges = sorted(goblins + elves)
        for i, row in enumerate(M):
            s += ''.join(row) + ' '
            for j in range(len(row)):
                for g in ges:
                    if g[0] == (i, j):
                        s += g[2] + '(' + str(g[1]) +') '
                
            s += '\n'
            
        print(s)
        
    def neighbors4(point): 
        "The four neighbors (without diagonals)."
        x, y = point
        return ((x-1, y), (x, y-1), (x, y+1), (x+1, y))
        
    def moves(x):
#         print([m for m in neighbors4(x) if M[m[0]][m[1]] == '.'])
        return [m for m in neighbors4(x) if M[m[0]][m[1]] == '.']

    goblins = [] 
    elves = []
    M = []
    for i, r in enumerate(rows):
        row = []
        for j, c in enumerate(r):
            row.append(c)
            if c == 'G':
                goblins.append([(i, j), 200, 'G'])
            if c == 'E':
                elves.append([(i, j), 200, 'E'])
        M.append(row)
    ges = sorted(goblins + elves)
    
#     m = ges[0]
#     enemies = [e for e in ges if e[2] != m[2]]
#     e = enemies[0]
#     print(m[0], '->' , e[0])
#     M[e[0][0]][e[0][1]] = '.'
#     return breadth_first(m[0], e[0], moves)

    def breadth_first(start, goals, moves_func):
        "Find a shortest sequence of states from start to the goal."
        frontier = deque([start]) # A queue of states
        previous = {start: None}  # start has no previous state; other states will
        all_paths = []
        while frontier:
            s = frontier.popleft()
            if s in goals:
                all_paths.append(path(previous, s))
            for s2 in moves_func(s):
                if s2 not in previous:
                    frontier.append(s2)
                    previous[s2] = s
        return all_paths

    def attack(m):
        targets = []
        for r, c in neighbors4(m[0]):
            if M[r][c] in 'GE' and M[r][c] != m[2]:
                for e in enemies:
                    if e[0] == (r, c):
                        targets.append(e)
        if targets:
            e = min(targets, key=lambda x: (x[1], x[0]))
            
            e[1] -= (power if m[2] == 'E' else 3)
#                 print(m, 'att', e)
            if e[1] <= 0:
#                     print('die', r,c , M[r][c], e)
                r, c = e[0]
#                 print('before')
#                 show()
                M[r][c] = '.'
#                 print('after')
#                 show()
                if e in goblins:
                    goblins.remove(e)
                if e in elves:
                    elves.remove(e)
                if e in enemies:
                    enemies.remove(e)
        return targets

#     show()
#     print()

    elves_count = len(elves)
        
    count = 0
    while True:
        count += 1
        ges = sorted(goblins + elves)
        for m in ges:
            if m not in goblins and m not in elves:
                continue
                
            enemies = [e for e in ges if e[2] != m[2] and e[1] > 0]
#             print('to consider', enemies)
            # Find shortest path to each enemy
#             print('I am ', m[2], m[0])

            if not attack(m):
#                 ges2 = sorted(goblins + elves)
#                 enemies = [e for e in ges if e[2] != m[2]]
#                 print('new ', enemies)
#                 for e in enemies:
# #                     print('find', m[0], e[0])
#                     # Note: move to near by targets, not the tartgets themselves
#                     for n in moves(e[0]):
#                         # Mark the target open first
# #                         M[e[0][0]][e[0][1]] = '.'
#                         apath = breadth_first(m[0], n, moves)
#                         if apath:
#                             paths += apath
#                         # Reset
# #                         M[e[0][0]][e[0][1]] = e[2]
# #                 print(m[0], e[0], paths)

                targets = []
                for e in enemies:
                    for n in moves(e[0]):
                        if n not in targets:
                            targets.append(n)
                
#                 print(targets)            
                paths = breadth_first(m[0], targets, moves)
                
                if paths:
                    shortest_dist = min(list(map(len, paths)))
                    target = min([p[-1] for p in paths if len(p) == shortest_dist])
                
                
                    # the first target in reading order is the chosen one
#                     target = min(p[-1] for p in paths)
#                     print('choose', target)

                    paths = breadth_first(m[0], [target], moves)
                    
                    if paths:
                        p = min(paths, key=lambda x: x[1])
#                         print('from', m[0], 'to', p)
                        
                        next_move = p[1]
                        # Make a move
                        old_pos = m[0]
                        M[old_pos[0]][old_pos[1]] = '.'
                        m[0] = next_move
                        M[next_move[0]][next_move[1]] = m[2]
        #                 print('...', m[2], old_pos, next_move)

                        attack(m)
    #             print()
          
            
#         if len(elves) < elves_count:
#             print('dead')
#             return
        
#         print('After', count, 'rounds')
#         show()
#         print()

            
        if not goblins or not elves:
            sumg = sum(g[1] for g in goblins)
            sume = sum(g[1] for g in elves)
            return count, count * (sumg + sume), (count - 1) * (sumg + sume)

rows = open('input15.txt').read().splitlines()
day15(rows)

In [ ]:
def day16(rows):
    def addr(regs, a, b, c): regs[c] = regs[a] + regs[b]
    def addi(regs, a, b, c): regs[c] = regs[a] + b
    def mulr(regs, a, b, c): regs[c] = regs[a] * regs[b]
    def muli(regs, a, b, c): regs[c] = regs[a] * b 
    def banr(regs, a, b, c): regs[c] = regs[a] & regs[b]
    def bani(regs, a, b, c): regs[c] = regs[a] & b 
    def borr(regs, a, b, c): regs[c] = regs[a] | regs[b]
    def bori(regs, a, b, c): regs[c] = regs[a] | b 
    def setr(regs, a, b, c): regs[c] = regs[a]
    def seti(regs, a, b, c): regs[c] = a
    def gtir(regs, a, b, c): regs[c] = 1 if regs[a] > regs[b] else 0
    def gtri(regs, a, b, c): regs[c] = 1 if regs[a] > b else 0
    def gtrr(regs, a, b, c): regs[c] = 1 if a > regs[b] else 0
    def eqir(regs, a, b, c): regs[c] = 1 if regs[a] == regs[b] else 0 
    def eqri(regs, a, b, c): regs[c] = 1 if regs[a] == b else 0
    def eqrr(regs, a, b, c): regs[c] = 1 if a == regs[b] else 0

    all_ops = [addr, addi, mulr, muli, banr, bani, borr, bori, setr, seti, gtir, gtri, gtrr, eqir, eqri, eqrr]
    
    def test(before, ops, after):
        count = 0
        for op in all_ops:
            regs = before.copy()
#             print(regs)
            op(regs, ops[1], ops[2], ops[3])
#             print(regs)
            if regs == after:
                count += 1
        return count >= 3

    idx = 0
    count = 0
    while idx < len(rows):
        before = get_ints(rows[idx])
        ops = get_ints(rows[idx + 1])
        after = get_ints(rows[idx + 2])
        idx += 4
#         print(before, ops, after)
        if test(before, ops, after):
            count += 1
    return count
    
day16(InputRows(16))

# Day 17: Reservoir Research
[Problem Description](https://adventofcode.com/2018/day/17)
Got absolutely no idea how to approach this problem. Need to get some ideas on reddit. Found a great solution from  `sciyoshi`, a recursive one like a flood fill problem. Still need to spend plenty of time to understand all the subtle details and write down a filling strategy.

In [ ]:
sys.setrecursionlimit(10000)

def day17(rows):
    clay = set()
    for row in rows:
        nums = get_ints(row)
        if row.index('x') < row.index('y'):
            for y in ints(nums[1], nums[2]):
                clay.add((nums[0], y))
        else:
            for x in ints(nums[1], nums[2]):
                clay.add((x, nums[0]))

    minx = min(clay, key=lambda x: x[0])[0]
    maxx = max(clay, key=lambda x: x[0])[0]
    miny = min(clay, key=lambda x: x[1])[1]
    maxy = max(clay, key=lambda x: x[1])[1]

    settled = set()
    flowing = set()

    def fill(p, dir):
        """
        Filling strategy:
            - First, fill myself. Then decide where's next.
            - Should I go down?
              -> Yes, if there's neither clay nor settled water below and still within the range.
            - Should I go to the sides? Note that filling is recursive, so all the below squares have been filled.
              -> No, if there's neither clay nor settled water below. Return.
              Now, there's something below.
              - Should I go left. Yes, if the left one is not clay
            - Should I go right? Similar to left.
            Note: as we fill in both directions, there will be overlaps. Need to check if a square is being filled then skip it.
            - Now, after both sides are filled, settle the water. Only apply for bottom filling.
            Water is settled if both sides are surrounded by clay.
            Need to ask the filling function to recursively let me know that whether
            the left/right bound has clay.
            Get all flowing water between the two clay bounds and settle it.
        """
        # First, fill myself
        flowing.add(p)

        # Should I go down?
        below = (p[0], p[1] + 1)

        # In this case: there's nothing below to support, water can only go down
        if below not in clay and below not in settled and 1 <= below[1] <= maxy:
            fill(below, (0, 1))

        # After filling down and there's no support below. Return.
        if below not in clay and below not in settled:
            return False

        # There's a support below, go both sides
        left = (p[0] - 1, p[1])
        right = (p[0] + 1, p[1])

        # Go left? Yes, if not clay and not being filled
        left_bounded = left in clay
        if left not in clay and left not in flowing:
            left_bounded = fill(left, (-1, 0))

        # Go right
        right_bounded = right in clay
        if right not in clay and right not in flowing:
            right_bounded = fill(right, (1, 0))

        # Settle water
        if dir == (0, 1) and left_bounded and right_bounded:
            settled.add(p)

            while left in flowing:
                settled.add(left)
                left = (left[0] - 1, left[1])

            while right in flowing:
                settled.add(right)
                right = (right[0] + 1, right[1])

        # Return whether the left fill is bounded, the right fill is bounded.
        return dir == (-1, 0) and left_bounded or dir == (1, 0) and right_bounded

    def show():
        s = ''
        for y in ints(miny, maxy):
            for x in ints(minx, maxx):
                if (x, y) == (500, miny):
                    s += '+'
                else:
                    s += '#' if (x, y) in clay else '.'
            s += '\n'
        f = open('out17.txt', 'w')
        f.write(s)

    # show()

    fill((500, 0), (0, 1))
    print(len([p for p in flowing | settled if miny <= p[1] <= maxy]))
    print(len([p for p in settled if miny <= p[1] <= maxy]))
    
day17(InputRows(17))

# Day 18: Settlers of The North Pole
[Problem Description](https://adventofcode.com/2018/day/18)

In [ ]:
def day16(rows):
    def addr(regs, a, b, c): regs[c] = regs[a] + regs[b]
    def addi(regs, a, b, c): regs[c] = regs[a] + b
    def mulr(regs, a, b, c): regs[c] = regs[a] * regs[b]
    def muli(regs, a, b, c): regs[c] = regs[a] * b 
    def banr(regs, a, b, c): regs[c] = regs[a] & regs[b]
    def bani(regs, a, b, c): regs[c] = regs[a] & b 
    def borr(regs, a, b, c): regs[c] = regs[a] | regs[b]
    def bori(regs, a, b, c): regs[c] = regs[a] | b 
    def setr(regs, a, b, c): regs[c] = regs[a]
    def seti(regs, a, b, c): regs[c] = a
    def gtrr(regs, a, b, c): regs[c] = 1 if regs[a] > regs[b] else 0
    def gtri(regs, a, b, c): regs[c] = 1 if regs[a] > b else 0
    def gtir(regs, a, b, c): regs[c] = 1 if a > regs[b] else 0
    def eqrr(regs, a, b, c): regs[c] = 1 if regs[a] == regs[b] else 0 
    def eqri(regs, a, b, c): regs[c] = 1 if regs[a] == b else 0
    def eqir(regs, a, b, c): regs[c] = 1 if a == regs[b] else 0

    all_ops = [addr, addi, mulr, muli, banr, bani, borr, bori, setr, seti, gtir, gtri, gtrr, eqir, eqri, eqrr]
    opmap = dict()
    
    def test(before, ops, after):
        count = 0
        newops = set()
        n = ops[0]
        for i, op in enumerate(all_ops):
            regs = before.copy()
#             print(regs)
            op(regs, ops[1], ops[2], ops[3])
#             print(regs)
            if regs == after:
                count += 1
                newops.add(i)
        if n not in opmap:
            opmap[n] = newops
        else:
            opmap[n] = opmap[n].intersection(newops)
            
        return count >= 3

    idx = 0
    count = 0
    while idx < len(rows):
        before = get_ints(rows[idx])
        ops = get_ints(rows[idx + 1])
        after = get_ints(rows[idx + 2])
        idx += 4
        if test(before, ops, after):
            count += 1
            
#     print(opmap)
#     print()
    
    def deduce(lookup):
        n = len(lookup)
        new_lookup = dict()
        while len(new_lookup) < n:
            lowest_key = min(lookup, key=(lambda k: len(lookup[k])))
            if len(lookup[lowest_key]) > 1:
                print('tricky case')
            else:
                x = lookup[lowest_key] = list(lookup[lowest_key])[0]
                new_lookup[lowest_key] =  x
                del lookup[lowest_key]
                for v in lookup.values():
                    if x in v:
                        v.remove(x)
        return new_lookup
    
    opmap = deduce(opmap)
    
    lines = open('test16.txt').read().splitlines()
    regs = [0,0,0,0]
    for line in lines:
        op, a, b, c = get_ints(line)
        all_ops[opmap[op]](regs, a, b, c)
    
    return regs[0]
    
day16(InputRows(16))

In [ ]:
def day18(rows):
    grid = defaultdict(str)
    
    for j, row in enumerate(rows):
        for i, c in enumerate(row):
            grid[(i, j)] = c
            
    for t in range(1000):
        tmp = defaultdict(str)
        for i in range(50):
            for j in range(50):
                c = (i, j)
                o = grid[c]
                if o == '.':
                    if len([n for n in neighbors8(c) if grid[n] == '|']) >= 3:
                        tmp[c] = '|'
                    else:
                        tmp[c] = '.'
                if o == '|':
                    if len([n for n in neighbors8(c) if grid[n] == '#']) >= 3:
                        tmp[c] = '#'
                    else:
                        tmp[c] = '|'
                if o == '#':
                    if len([n for n in neighbors8(c) if grid[n] == '#']) and len([n for n in neighbors8(c) if grid[n] == '|']):
                        tmp[c] = '#'
                    else:
                        tmp[c] = '.'
        grid = tmp
        tc = 0
        lc = 0
        for i in range(50):
            for j in range(50):
                c = (i, j)
                if grid[c] == '|':
                    tc += 1
                if grid[c] == '#':
                    lc += 1
        print(t + 1, tc, lc, tc * lc)
    
day18(InputRows(18))

In [ ]:
1000 % 28

In [ ]:
1000000000 % 28

The cycle is 28, so just get the mod. Accidentally, it's the same as 1000.

# Day 19: Go With The Flow
[Problem Description](https://adventofcode.com/2018/day/19)

In [ ]:
def day19(ins):
    def addr(regs, a, b, c): regs[c] = regs[a] + regs[b]
    def addi(regs, a, b, c): regs[c] = regs[a] + b
    def mulr(regs, a, b, c): regs[c] = regs[a] * regs[b]
    def muli(regs, a, b, c): regs[c] = regs[a] * b 
    def banr(regs, a, b, c): regs[c] = regs[a] & regs[b]
    def bani(regs, a, b, c): regs[c] = regs[a] & b 
    def borr(regs, a, b, c): regs[c] = regs[a] | regs[b]
    def bori(regs, a, b, c): regs[c] = regs[a] | b 
    def setr(regs, a, b, c): regs[c] = regs[a]
    def seti(regs, a, b, c): regs[c] = a
    def gtrr(regs, a, b, c): regs[c] = 1 if regs[a] > regs[b] else 0
    def gtri(regs, a, b, c): regs[c] = 1 if regs[a] > b else 0
    def gtir(regs, a, b, c): regs[c] = 1 if a > regs[b] else 0
    def eqrr(regs, a, b, c): regs[c] = 1 if regs[a] == regs[b] else 0 
    def eqri(regs, a, b, c): regs[c] = 1 if regs[a] == b else 0
    def eqir(regs, a, b, c): regs[c] = 1 if a == regs[b] else 0
        
    cmds = {
      'addr': addr, 'addi': addi,
      'mulr': mulr, 'muli': muli,
      'banr': banr, 'bani': bani,
      'borr': borr, 'bori': bori,
      'setr': setr, 'seti': seti,
      'gtrr': gtrr, 'gtri': gtri, 'gtir': gtir,
      'eqrr': eqrr, 'eqri': eqri, 'eqir': eqir
    }

    idx = 1
    ip = 0
    regs = [0,0,0,0,0,0]
    
    while 0 <= ip < len(ins):
        r = ins[ip]
        a, b, c = get_ints(r)
        regs[idx] = ip
        cmds[r[:4]](regs, a, b, c)
        ip = regs[idx] + 1
        
    return regs[0]

day19(InputRows(19))

# Day 20: A Regular Map
[Problem Description](https://adventofcode.com/2018/day/20)

In [ ]:
def day20(line):
    def combine(routes1, routes2, routes3):
        routes = []
        for r1 in routes1:
            for r2 in routes2:
                for r3 in routes3:
                    routes.append(r1 + r2 + r3)
        return routes

    def go(line):
        bracket = line.find('(')
        option = line.find('|')

        if bracket == -1 and option == -1:
            return [line]

        if option == -1 or option != -1 and bracket != -1 and bracket < option:
            # Where's the matching closing bracket?
            idx = bracket
            depth = 0
            while True:
                if line[idx] == '(':
                    depth += 1
                if line[idx] == ')':
                    depth -= 1
                    if depth == 0:
                        break
                idx += 1

            return combine(go(line[:bracket]), go(line[bracket+1:idx]), go(line[idx+1:]))

        if bracket == -1 or option != -1 and bracket != -1 and option < bracket:
            return go(line[:option]) + go(line[option+1:])

    def build_graph(routes):
        g = defaultdict(set)
        dirs = {
            'E': (0, 1),
            'W': (0, -1),
            'N': (-1, 0),
            'S': (1, 0)
        }

        for r in routes:
            p = (0, 0)
            for c in r:
                x, y = dirs[c]
                p2 = p[0] + x, p[1] + y
                g[p].add(p2)
                g[p2].add(p)
                p = p2

        return g
    
    def moves(n):
        return graph[n]
    
    def get_all_shortest_paths(g):
        nodes = g.keys()
        s = (0, 0)
        return max(len(breadth_first(s, n, moves)) - 1 for n in nodes)
#         for n in nodes:
#             print(s, n, breadth_first(s, n, moves))

    routes = go(line)
#     print('routes', routes)
    graph = build_graph(routes)
    print(len(sorted(graph.keys())))
    return get_all_shortest_paths(graph)

print(day20('WNE') == 3)
print(day20('ENWWW(NEEE|SSE(EE|N))') == 10)
print(day20('ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN') == 18)
print(day20(InputString(20)[1:-1]))

In [ ]:
'123'.find('x')

# Day 21: Chronal Conversion
[Problem Description](https://adventofcode.com/2018/day/21)

In [ ]:
def day21(ins):
    def addr(regs, a, b, c): regs[c] = regs[a] + regs[b]
    def addi(regs, a, b, c): regs[c] = regs[a] + b
    def mulr(regs, a, b, c): regs[c] = regs[a] * regs[b]
    def muli(regs, a, b, c): regs[c] = regs[a] * b 
    def banr(regs, a, b, c): regs[c] = regs[a] & regs[b]
    def bani(regs, a, b, c): regs[c] = regs[a] & b 
    def borr(regs, a, b, c): regs[c] = regs[a] | regs[b]
    def bori(regs, a, b, c): regs[c] = regs[a] | b 
    def setr(regs, a, b, c): regs[c] = regs[a]
    def seti(regs, a, b, c): regs[c] = a
    def gtrr(regs, a, b, c): regs[c] = 1 if regs[a] > regs[b] else 0
    def gtri(regs, a, b, c): regs[c] = 1 if regs[a] > b else 0
    def gtir(regs, a, b, c): regs[c] = 1 if a > regs[b] else 0
    def eqrr(regs, a, b, c): regs[c] = 1 if regs[a] == regs[b] else 0 
    def eqri(regs, a, b, c): regs[c] = 1 if regs[a] == b else 0
    def eqir(regs, a, b, c): regs[c] = 1 if a == regs[b] else 0
        
    cmds = {
      'addr': addr, 'addi': addi,
      'mulr': mulr, 'muli': muli,
      'banr': banr, 'bani': bani,
      'borr': borr, 'bori': bori,
      'setr': setr, 'seti': seti,
      'gtrr': gtrr, 'gtri': gtri, 'gtir': gtir,
      'eqrr': eqrr, 'eqri': eqri, 'eqir': eqir
    }

    idx = 1
    ip = 0
    regs = [0,0,0,0,0,0]
    seen = set()
    
    while True:
    
        while 0 <= ip < len(ins):
            r = ins[ip]
            a, b, c = get_ints(r)
            regs[idx] = ip
            cmds[r[:4]](regs, a, b, c)

            if ip == 28:
#                 print(regs[2])
                if regs[2] in seen:
                    print('part 2', regs[2])
                seen.add(regs[2])
            ip = regs[idx] + 1
        
    return regs[0]

day21(InputRows(21))

In [ ]:
def day21():
    part1 = False
    c = 0
    seen = set()
    last = None

    while True:
        f = c | 65536
        c = 4843319
        
        while True:
            e = f & 255
            c += e
            c &= 16777215
            c *= 65899
            c &= 16777215

            if 256 > f:
                if part1:
                    print('part 1', c)
                    return
                else:
                    if c in seen:
                        print('part 2', last)
                        return
                    else:
                        last = c
                        seen.add(c)
                        break
            else:
                f = f // 256
            
day21()

# Day 22: Mode Maze
[Problem Description](https://adventofcode.com/2018/day/22)

Part 1 gave me the best place ever: 108.

In [ ]:
def day22(depth, target):
    levels = defaultdict(int)
    total = 0
    
    def compute_index(c):
        x, y = c
        if c == (0, 0) or c == target: return 0
        if y == 0: return x * 16807
        if x == 0: return y * 48271
        return levels[(x - 1, y)] * levels[(x, y - 1)]
    
    for i in ints(0, target[0]):
        for j in ints(0, target[1]):
            c = (i, j)
            idx = compute_index(c)
            levels[c] = (idx + depth) % 20183
            total += levels[c] % 3
            
    print('part 1', total)
    
    def typ(c):
        return levels[c] % 3
    
    # Dijkstra's algorithm. A state is a combination of location and equipment.
    # torch, climbing, neither = 1, 2, 0
    # rocky, wet, narrow = 0, 1, 2
    # set constants in a way that matching numbers means invalid
    
    # Extend the grid first
    w = target[0] + 100
    h = target[1] + 100
    for i in ints(0, w):
        for j in ints(0, h):
            c = (i, j)
            idx = compute_index(c)
            levels[c] = (idx + depth) % 20183
    
    start = (0, 0)
    frontier = [(0, start, 1)]
    costs = {}

    while frontier:
        d, c, t = heappop(frontier)
        if (c, t) == (target, 1):
            print('part 2', d)
            return
        
        # Switching equipment can be a new state
        states = []
        for t2 in range(3):
            if t2 != typ(c):
                states.append((d + 7, c, t2))
        
        # Each direction can be a new state
        for n in neighbors4(c):
            if n[0] < 0 or n[1] < 0 or n[0] > w or n[1] > h: continue
            
            if t != typ(n):
                states.append((d + 1, n, t))
        
        # Add states
        for s in states:
            s2 = (s[1], s[2])
            if s2 not in costs or s[0] < costs[s2]:
                costs[s2] = s[0]
                heappush(frontier, s)
                
day22(9171, (7, 721))

# Day 23: Experimental Emergency Teleportation
[Problem Description](https://adventofcode.com/2018/day/23)

I have no idea how to approach part 2 as the search space is massive. There are two approaches in reddit: use an SMT solver (`z3` library) to find an optimized solution or use a heuristic. Below is the code for a subdivision heuristic from reddit. It just happened to give the correct answer. Also, playing around with the starting location and step size could lead to different locations but most of them have the same Manhattan distance to the origin!

In [ ]:
def mht(p1, p2):
    return sum(abs(p1[i] - p2[i]) for i in range(3))
    
def parse_data(rows):
    robots = []
    for row in rows:
        x, y, z, r = get_ints(row)
        robots.append(((x, y, z), r))
    return robots

def find_in_range(robots):
    loc, rad = max(robots, key=lambda x: x[1])
    return sum(mht(loc, c) <= rad for c, _ in robots)
    
def find_shortest(robots):
    def count(p):
        return sum(mht(p, c) <= r for c, r in robots)
    
    xs = [r[0][0] for r in robots]
    ys = [r[0][1] for r in robots]
    zs = [r[0][2] for r in robots]
    
    # Start with the center
    n = len(robots)
    center = (sum(xs) // n, sum(ys) // n, sum(zs) // n)
    
    size = max([max(xs) - min(xs), max(ys) - min(ys), max(zs) - min(zs)])
    dist = 1
    while dist < size:
        dist *= 2
    
    xs = [center[0] - dist, center[0] + dist]
    ys = [center[1] - dist, center[1] + dist]
    zs = [center[2] - dist, center[2] + dist]
    dist //= 2

    while True:
        max_loc = None
        max_count = -1
        min_dist = float('inf')

        for x in ints(min(xs), max(xs), dist):
            for y in ints(min(ys), max(ys), dist):
                for z in ints(min(zs), max(zs), dist):
                    p = (x, y, z)
                    bots = count(p)
                    d = mht(p, (0, 0, 0))
                    if bots > max_count or bots == max_count and d < min_dist:
                        max_loc = p
                        max_count = bots
                        min_dist = d

        if dist == 1:
            return min_dist
        else:            
            xs = [max_loc[0] - dist, max_loc[0] + dist]
            ys = [max_loc[1] - dist, max_loc[1] + dist]
            zs = [max_loc[2] - dist, max_loc[2] + dist]
            dist //= 2
    
def day23(rows):
    robots = parse_data(rows)
    part1 = find_in_range(robots)
    part2 = find_shortest(robots)    
    return part1, part2

day23(InputRows(23))

# Day 24: Immune System Simulator 20XX
[Problem Description](https://adventofcode.com/2018/day/24)

A battle simulation like Day 15; however, it's easier as no path finding involved.
- I build a proper class to handle the units instead of tuples to avoid messing around.
- I hard-code the input instead of writing a proper parser as a I see the input is short and the parser is a bit complicated. I should up my game of text processing and/or regex.
- For the submission, I did a manual binary search to find the answer for part 2 :).

In [ ]:
class Group:
    def __init__(self, size, hp, weaks, immunes, damage, damage_type, initiative):
        self.size = size
        self.hp = hp
        self.weaks = weaks
        self.immunes = immunes
        self.damage = damage
        self.damage_type = damage_type
        self.initiative = initiative
        self.power = self.size * self.damage

    def __repr__(self):
        return 'name={}, size={}, hp={}, weaks={}, immunes={}, damage_type={}, damage={}, initiative={}, power={}'.format(
            self.name, self.size, self.hp, self.weaks, self.immunes, self.damage_type, self.damage, self.initiative, self.power)

    def set_name(self, army, index):
        self.army = army
        self.name = army + ' ' + str(index + 1)

    def compute_damage(self, g):
        if self.damage_type in g.immunes: return 0
        if self.damage_type in g.weaks: return 2 * self.power
        return self.power

    def set_target(self, target):
        self.target = target

    def update_battle(self, deaths):
        self.size = max(0, self.size - deaths)
        self.power = self.size * self.damage

    def boost_damage(self, boost):
        self.damage += boost
        self.power = self.size * self.damage

In [ ]:
def get_data(boost = 0):
    # Example
#         immune = [
#             Group(17, 5390, ['radiation', 'bludgeoning'], [], 4507, 'fire', 2),
#             Group(989, 1274, ['bludgeoning', 'slashing'], ['fire'], 25, 'slashing', 3)
#         ]

#         infection = [
#             Group(801, 4706, ['radiation'], [], 116, 'bludgeoning', 1),
#             Group(4485, 2961, ['fire', 'cold'], ['radiation'], 12, 'slashing', 4)
#         ]

    # Real input
    immune = [
        Group(479, 3393, ['radiation'], [], 66, 'cold', 8),
        Group(2202, 4950, ['fire'], ['slashing'], 18, 'cold', 2),
        Group(8132, 9680, ['bludgeoning', 'fire'], ['slashing'], 9, 'radiation', 7),
        Group(389, 13983, [], ['bludgeoning'], 256, 'cold', 13),
        Group(1827, 5107, [], [], 24, 'slashing', 18),
        Group(7019, 2261, [], ['radiation', 'slashing', 'cold'], 3, 'fire', 16),
        Group(4736, 8421, ['cold'], [], 17, 'slashing', 3),
        Group(491, 3518, ['cold'], ['fire', 'bludgeoning'], 65, 'radiation', 1),
        Group(2309, 7353, [], ['radiation'], 31, 'bludgeoning', 20),
        Group(411, 6375, ['cold', 'fire'], ['slashing'], 151, 'bludgeoning', 14)
    ]

    infection = [
        Group(148, 31914, ['bludgeoning'], ['radiation', 'cold', 'fire'], 416, 'cold', 4),
        Group(864, 38189, [], [], 72, 'slashing', 6),
        Group(2981, 7774, [], ['bludgeoning', 'cold'], 4, 'fire', 15),
        Group(5259, 22892, [], [], 8, 'fire', 5),
        Group(318, 16979, ['fire'], [], 106, 'bludgeoning', 9),
        Group(5017, 32175, ['slashing'], ['radiation'], 11, 'bludgeoning', 17),
        Group(4308, 14994, ['slashing'], ['fire', 'cold'], 5, 'fire', 10),
        Group(208, 14322, ['radiation'], [], 133, 'cold', 19),
        Group(3999, 48994, ['cold', 'slashing'], [], 20, 'cold', 11),
        Group(1922, 34406, ['slashing'], [], 35, 'slashing', 12)
    ]

    for i, g in enumerate(immune):
        g.set_name('Immune', i)
        g.boost_damage(boost)

    for i, g in enumerate(infection):
        g.set_name('Infection', i)

    return immune + infection

In [ ]:
def battle(all_groups):
    verbose = False
    
    while True:
        # Info:
        if verbose:
            count += 1
            print('Fight', count)

        groups = [g for g in all_groups if g.size > 0]

        if verbose:
            for g in groups:
                if g.size > 0:
                    print('-', g.name, g.size)

        # Selection
        chosen = set()
        for g in sorted(groups, key=lambda x: (x.power, x.initiative), reverse=True):
            # Select group that would deal the most damage, largest power, highest initiative
            enemies = [g2 for g2 in groups if g2.army != g.army and g.compute_damage(g2) > 0 and g2 not in chosen]
            enemies.sort(key=lambda e: (g.compute_damage(e), e.power, e.initiative), reverse=True)

            # Pick the first one
            if enemies:
                e = enemies[0]
                chosen.add(e)
                g.set_target(e)
                if verbose:
                    print(g.name, '...', e.name, g.compute_damage(e))
            else:
                g.set_target(None)

        # Attack
        keep_battling = False

        for g in sorted(groups, key=lambda x: x.initiative, reverse=True):
            if not g.target: continue

            deaths = g.compute_damage(g.target) // g.target.hp

            if verbose:
                print(g.name, '>>>', g.target.name, 'kill', min(deaths, g.target.size))

            # Update the deaths
            g.target.update_battle(deaths)

            if deaths > 0:
                keep_battling = True

        if verbose:
            print()

        if not keep_battling:
            left = [g for g in groups if g.size > 0]
            return set(g.army for g in left), sum(g.size for g in left)

In [ ]:
def day24():
    part1 = battle(get_data())[1]
    
    for b in range(1000):
        r = battle(get_data(b))
        if r[0] == {'Immune'}:
            part2 = r[1]
            break
            
    return part1, part2

day24()

# Day 25: Four-Dimensional Adventure
[Problem Description](https://adventofcode.com/2018/day/25)

This is to find the number of connected components in a graph. I first forgot to merge two components if a point belongs to both.

In [ ]:
def day25(rows):
    def mht(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1]) + abs(p1[2] - p2[2]) + abs(p1[3] - p2[3])
    
    points = [tuple(get_ints(r)) for r in rows]
    
    constells = []
    for p in points:
        ec = None
        to_remove = []
        
        # If found a constellation, set it to ec. If existed, merge with ec.
        for c in constells:
            if any(mht(p, p1) <= 3 for p1 in c):
                if ec:
                    # Merge
                    ec.update(c)
                    to_remove.append(c)
                else:
                    # Set
                    c.add(p)
                    ec = c
        if ec:
            for c in to_remove:
                constells.remove(c)
        else:
            # Add new
            constells.append({p})
            
    return len(constells)
    
day25(InputRows(25))